In [1]:
# import greenery_index_results_big.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
import sys
from pathlib import Path
import folium
from folium.plugins import HeatMap

In [ ]:
data_path = Path('~/Desktop/delft_download/data/Rotterdam_NL')
panoids = gpd.read_file(data_path / 'panoids/panoids.geojson')
print(f"Loaded panoids data with {len(panoids)} entries.")

# Load the data
data = pd.read_csv('../Data/greenery_index_results_big.csv')
data['Greenery_Index'] = data['Greenery_Index'].str.strip('[]').astype(float)
# Check the first few rows "Image_Name" (data) and 
data.head()

In [ ]:
# Create a column to preserve the original order of panoids
panoids['original_index'] = panoids.index

# Melt the DataFrame to get one row per image
df_melted = panoids.melt(
    id_vars=['panoid', 'year', 'month', 'owner', 'lat', 'lng', 'ask_lng', 'ask_lat', 'consulted', 'dist', 'angle', 'geometry', 'original_index'],
    value_vars=['im_side_a', 'im_front', 'im_side_b', 'im_back'],
    var_name='image_type',
    value_name='image_name'
)

# Create a column for sorting by the original order within each panoid
df_melted['order'] = df_melted['image_type'].map({
    'im_side_a': 0, 'im_front': 1, 'im_side_b': 2, 'im_back': 3
})

# Sort the DataFrame by the original index of panoids and the new order column
df_melted = df_melted.sort_values(by=['original_index', 'order']).drop(columns=['original_index', 'order']).reset_index(drop=True)

# Check the transformed and sorted DataFrame
df_melted.head()


In [ ]:
# join the data with the panoids on Image_Name (data) and image_name (panoids) columns
# make sure to only keep one panoid column and one image_name column
df = pd.merge(data, df_melted, left_on='Image_Name', right_on='image_name', how='inner').drop(columns=['Image_Name', 'panoid_y'])
df = df.rename(columns={'panoid_x': 'panoid'})

df.head()

In [ ]:
df.year.value_counts()

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# Normalize Greenery_Index for color scaling
norm = colors.Normalize(vmin=df['Greenery_Index'].min(), vmax=df['Greenery_Index'].max())
colormap = cm.ScalarMappable(norm=norm, cmap='YlGn')

# Initialize the map centered around the mean latitude and longitude
map_center = [df['lat'].mean(), df['lng'].mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Add circle markers to the map
for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['lat'], row['lng']),
        radius=row['Greenery_Index'] * 10,  # Scale radius
        color=colors.to_hex(colormap.to_rgba(row['Greenery_Index'])),
        fill=True,
        fill_color=colors.to_hex(colormap.to_rgba(row['Greenery_Index'])),
        fill_opacity=0.7,
        popup=f'Greenery Index: {row["Greenery_Index"]}'
    ).add_to(m)

# Save and display the map
m.save("greenery_circle_markers.html")

# If running in a Jupyter Notebook, display the map inline
m